In [1]:
import pandas as pd
colums = ['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo', 'especie']

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/afoone/caipc-laberit-oct2022/master/data/Iris.csv',header=0, names=colums)

In [3]:
df.shape

(150, 5)

In [4]:
df['especie'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: especie, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
df

,l_sepal,a_sepal,l_petalo,a_petalo,especie
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
train , test = train_test_split(df, test_size=0.20, shuffle=True, random_state=0)

In [8]:
train.shape

(90, 5)

In [9]:
train['especie'].value_counts()

Iris-setosa        34
Iris-virginica     29
Iris-versicolor    27
Name: especie, dtype: int64

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
model = DecisionTreeClassifier(criterion="gini")

# Poda 1: por profundidad
#model = DecisionTreeClassifier(criterion="gini", max_depth=1)

# Poda 2: por mínimo de muestras para seguir dividiendo
# model = DecisionTreeClassifier(criterion="gini", min_samples_split=30)

# hay más...

In [54]:
df.columns

Index(['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo', 'especie'], dtype='object')

In [55]:
X = train[['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo']]

In [56]:
y = train['especie']

In [57]:
model.fit(X, y)

DecisionTreeClassifier()

In [58]:
test['prediction'] = model.predict(test[['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo']])

In [59]:
test.head(10)

,l_sepal,a_sepal,l_petalo,a_petalo,especie,prediction
115,5.8,2.8,5.1,2.4,Iris-virginica,Iris-virginica
63,6.0,2.2,4.0,1.0,Iris-versicolor,Iris-versicolor
34,5.5,4.2,1.4,0.2,Iris-setosa,Iris-setosa
108,7.3,2.9,6.3,1.8,Iris-virginica,Iris-virginica
8,5.0,3.4,1.5,0.2,Iris-setosa,Iris-setosa
101,6.3,3.3,6.0,2.5,Iris-virginica,Iris-virginica
41,5.0,3.5,1.3,0.3,Iris-setosa,Iris-setosa
87,6.7,3.1,4.7,1.5,Iris-versicolor,Iris-versicolor
77,6.8,2.8,4.8,1.4,Iris-versicolor,Iris-versicolor
72,6.1,2.8,4.0,1.3,Iris-versicolor,Iris-versicolor


In [63]:
len(test[test['especie']==test['prediction']]) / len(test)

0.95

In [64]:
pd.crosstab(
    test['especie'],
    test['prediction'],
    rownames=['real'],
    colnames=['prediccion']
)

prediccion,Iris-setosa,Iris-versicolor,Iris-virginica
real,,,
Iris-setosa,16,0,0
Iris-versicolor,0,22,1
Iris-virginica,0,2,19


# Visualizar árbol de decisión

In [65]:
from sklearn.tree import export_graphviz

In [66]:
with open('./tree.dot', 'w') as dotfile:
    export_graphviz(model, out_file=dotfile, feature_names=['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo'])
    dotfile.close()

In [67]:
import os 
from graphviz import Source
file = open('./tree.dot', 'r')
text = file.read()
Source(text)


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [68]:
from sklearn.model_selection import KFold

In [69]:
df

,l_sepal,a_sepal,l_petalo,a_petalo,especie
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


In [71]:
X2 = df[['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo']]
y2 = df['especie']

In [79]:
kf = KFold(n_splits=5, shuffle=True, random_state=88)

In [80]:
kf.n_splits

5

In [82]:
for train_index, test_index in kf.split(X2):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    est = DecisionTreeClassifier(criterion='gini')
    est.fit(X_train, y_train)
    prediction = est.predict(X_test)
    print(len(y_test == prediction)/len(y_test))

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  16  18  21  23
  25  26  27  28  29  31  32  33  34  35  36  37  38  40  41  42  43  44
  45  46  47  48  49  50  51  54  56  58  59  61  62  63  64  65  67  68
  69  70  71  72  74  75  76  77  78  79  80  81  82  83  85  86  87  88
  89  90  91  92  93  94  96  97  98  99 101 102 103 104 105 106 107 108
 110 112 114 115 118 119 122 124 125 126 127 128 130 131 132 133 135 136
 137 138 139 141 142 143 144 145 146 147 148 149] TEST: [ 14  15  17  19  20  22  24  30  39  52  53  55  57  60  66  73  84  95
 100 109 111 113 116 117 120 121 123 129 134 140]


KeyError: "None of [Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,\n            ...\n            139, 141, 142, 143, 144, 145, 146, 147, 148, 149],\n           dtype='int64', length=120)] are in the [columns]"

# Random forest

In [83]:
from sklearn.ensemble import RandomForestClassifier

In [85]:
model = RandomForestClassifier(n_estimators=10, criterion="gini", n_jobs=3, oob_score=True)

In [86]:
model.fit(X, y)

/Users/atienda/miniforge3/envs/caipcoctubre/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(n_estimators=10, n_jobs=3, oob_score=True)

In [87]:
model.predict(X)

array(['Iris-versicolor', 'Iris-setosa', 'Iris-virginica',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-setosa', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica',
       'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versic

In [89]:
test['predict'] = model.predict(
    test[['l_sepal', 'a_sepal', 'l_petalo', 'a_petalo']]
)

In [90]:
test

,l_sepal,a_sepal,l_petalo,a_petalo,especie,prediction,predict
115,5.8,2.8,5.1,2.4,Iris-virginica,Iris-virginica,Iris-virginica
63,6.0,2.2,4.0,1.0,Iris-versicolor,Iris-versicolor,Iris-versicolor
34,5.5,4.2,1.4,0.2,Iris-setosa,Iris-setosa,Iris-setosa
108,7.3,2.9,6.3,1.8,Iris-virginica,Iris-virginica,Iris-virginica
8,5.0,3.4,1.5,0.2,Iris-setosa,Iris-setosa,Iris-setosa
101,6.3,3.3,6.0,2.5,Iris-virginica,Iris-virginica,Iris-virginica
41,5.0,3.5,1.3,0.3,Iris-setosa,Iris-setosa,Iris-setosa
87,6.7,3.1,4.7,1.5,Iris-versicolor,Iris-versicolor,Iris-versicolor
77,6.8,2.8,4.8,1.4,Iris-versicolor,Iris-versicolor,Iris-versicolor
72,6.1,2.8,4.0,1.3,Iris-versicolor,Iris-versicolor,Iris-versicolor


In [92]:
(test['predict']!=test['especie']).sum()

7

In [93]:
model.oob_score_

0.9333333333333333